In [1]:
from __future__ import division, print_function
import os
import numpy as np
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import pickle
import operator

In [2]:
app = Flask(__name__)

dataset = []
def loadDataset(filename):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break

loadDataset("my.dat")

In [3]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    
    #Method to calculate distance between two instances.
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    print("distane is",distance)
    return distance


In [4]:
def getNeighbors(trainingSet , instance , k):
    distances =[]
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    print("neighbors is ",neighbors)
    return neighbors   

In [5]:
def nearestClass(neighbors):
    classVote ={}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1 
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    print("sorter is ",sorter)
    return sorter[0][0]

In [6]:
print('Model loaded. Check http://127.0.0.1:5000/')

Model loaded. Check http://127.0.0.1:5000/


In [7]:

@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('music.html')

In [8]:
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['image']
        
        # Save the file to ./uploads
        basepath = "C:/Externship/Flask"
        file_path = os.path.join(basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        print(file_path)
        i=1
        
        results = {1: 'blues', 2: 'classical', 3: 'country', 4: 'disco', 5: 'hiphop', 
                   6: 'jazz', 7: 'metal', 8: 'pop', 9: 'reggae', 10: 'rock'}
    
        (rate,sig)=wav.read(file_path)
        print(rate,sig)
        mfcc_feat=mfcc(sig,rate,winlen=0.020,appendEnergy=False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature=(mean_matrix,covariance,0)
        pred=nearestClass(getNeighbors(dataset ,feature , 8))
        
        print("predicted genre = ",pred,"class = ",results[pred])
        return "This song is classified as a "+str(results[pred])

In [9]:

if __name__ == '__main__':
    app.run(threaded = False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Aug/2023 14:21:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2023 14:21:43] "GET /favicon.ico HTTP/1.1" 404 -


C:/Externship/Flask\uploads\disco.00003.wav
22050 [-11036  -6158  -7126 ...  -4961  -4266  -3712]
distane is 36.862531331257905
distane is 28.11549979115233
distane is 53.92814198028502
distane is 28.323902782581776
distane is 39.57569807827844
distane is 37.28875038074472
distane is 44.97088723521253
distane is 49.583517895662446
distane is 66.57254778566798
distane is 65.62322217565033
distane is 74.94827618377718
distane is 51.385371052476906
distane is 70.61378443127407
distane is 53.45855475514557
distane is 72.92512509058503
distane is 49.85401076487701
distane is 45.49011329164947
distane is 38.424479596508014
distane is 67.5350761098737
distane is 95.31994420989906
distane is 64.68634991773881
distane is 57.36586438261233
distane is 70.19638450600002
distane is 54.065129523781536
distane is 53.536882925043635
distane is 89.23945008098553
distane is 58.62383551126385
distane is 225.9215694241114
distane is 67.22999182080684
distane is 130.26981925666777
distane is 60.58066642405

127.0.0.1 - - [02/Aug/2023 14:22:12] "POST /predict HTTP/1.1" 200 -


distane is 36.30619886773252
distane is 42.69902325771362
distane is 49.918470273375675
distane is 48.774757841423344
distane is 29.384763653830923
distane is 30.179656295869393
distane is 52.533219778155825
distane is 24.438420230709006
distane is 51.30331439150764
distane is 35.594300552177444
distane is 60.233979206845845
distane is 32.21923644469521
distane is 51.52575157775372
distane is 75.04375963260946
distane is 36.241168200878114
distane is 31.315235115592408
distane is 79.02812801060546
distane is 29.84606092216837
distane is 46.79824019789758
distane is 27.692673747671734
distane is 39.89141468070941
distane is 38.30662710707353
distane is 77.16801426457967
distane is 42.675352990597005
distane is 59.73739136974541
distane is 34.19608733947404
distane is 28.24401944323293
distane is 27.70374707813454
distane is 32.85225970649281
distane is 31.75467823906031
distane is 32.51899972444277
distane is 39.07108033626979
distane is 28.312765059632007
distane is 35.79891451330289
d